## <center>Nantucket

Nantucket.ipynb, the interactive command-centre notebook: when run, it defines the Moby-Dick-themed project structure directly in code, writes a matching YAML manifest, scaffolds any missing folders and placeholder notebooks to bring your OneDrive into alignment, validates that the actual directory structure matches the declared plan, and regenerates human-readable documentation: a Markdown table and Mermaid diagram that appear in both GitHub and OneNote.
Because the entire Leviathan repository is cloned into Databricks Repos, Spouter_inn.ipynb travels with it. You can open and run the notebook in Azure Databricks to scaffold, validate, and document the same structure there. Edit the structure in the notebook, rerun the cells, and every environment—OneDrive, GitHub, Databricks—stays in lockstep.

Create all folders in a YAML file

In [54]:
import yaml, pathlib, json

NOTEBOOK_DIR = pathlib.Path.cwd()
ROOT         = NOTEBOOK_DIR.parent
MANIFEST     = NOTEBOOK_DIR / "leviathan_directory_manifest.yml"

# --- 1. Moby-Dick themed structure (unchanged core) ---------------------------
structure = {
    "Leviathan": {
        "type": "directory",
        "metaphor": "Leviathan",
        "role": "Project root",
        "notes": "",
        "children": {

            # ───────── docs / planning ─────────
            "Pequod": {
                "type": "directory", "metaphor": "Pequod",
                "role": "Docs & notebooks hub", "notes": "",
                "children": {
                    "Chartroom": {
                        "type": "directory", "metaphor": "Chartroom",
                        "role": "High-level overviews + config", "notes": "",
                        "children": {
                            "Maps":    {"type": "directory","metaphor":"Maps",    "role":"Architecture diagrams + specs","notes":"","children":{}},
                            "Logbook": {"type": "directory","metaphor":"Logbook", "role":"Project diary & notes",        "notes":"","children":{}},
                        },
                    },
                },
            },

            # Nantucket – this notebook & outputs
            "Nantucket": {
                "type": "directory", "metaphor": "Nantucket",
                "role": "Planning & docs hub (this notebook)", "notes": "",
                "children": {
                    f: {"type": "file", "metaphor": "", "role": "", "notes": ""}
                    for f in [
                        "Spouter_inn.ipynb",
                        "leviathan_directory_manifest.yml",
                        "Leviathan_structure_source.pptx",
                        "Leviathan_structure_A4.png"
                    ]
                },
            },

            # ───────── raw data lake ───────────
            "Ocean": {
                "type": "directory", "metaphor": "Ocean",
                "role": "Data lake", "notes": "",
                "children": {
                    "England_1": {"type":"directory","metaphor":"England_1","role":"Premier League data","notes":"","children":{}},
                    "England_2": {"type":"directory","metaphor":"England_2","role":"Championship data","notes":"","children":{}},
                },
            },

            # ───────── feature engineering ─────
            "Armory": {
                "type": "directory", "metaphor": "Armory",
                "role": "Feature-engineering pipelines", "notes": "",
                "children": {
                    "Gam": {"type":"directory","metaphor":"Gam","role":"Cross-league utilities","notes":"","children":{}},
                    "England_1": {
                        "type": "directory", "metaphor": "England_1",
                        "role": "League-specific transforms", "notes": "",
                        "children": {
                            n: {"type":"directory","metaphor":n,"role":"","notes":"","children":{}}
                            for n in ["Blubber_room","Crows_nest","Whaleboat","Tackle","Line","Harpoon"]
                        },
                    },
                },
            },

            # ───────── model staging ───────────
            "Awhalin": {
                "type": "directory", "metaphor": "Awhalin",
                "role": "Model staging", "notes": "",
                "children": {
                    "Gam": {"type":"directory","metaphor":"Gam","role":"","notes":"","children":{}},
                    "England_1": {
                        "type": "directory", "metaphor": "England_1",
                        "role": "", "notes": "",
                        "children": {
                            n: {"type":"directory","metaphor":n,"role":"","notes":"","children":{}}
                            for n in [
                                "Mapple","Peleg","Bildad","Flask","Stubb","Fedallah","Tashtego",
                                "Daggoo","Elijah","Ahab","Starbuck","Ishmael","Queequeg","Rachel"
                            ]
                        },
                    },
                },
            },
        },
    }
}


# --- 2. helper to add scripts/maps/outputs in selected branches ---------------
def _inject_std(node):
    node["children"].update({
        "scripts": {"type": "directory","metaphor":"","role":"","notes":"","children": {}},
        "maps":    {"type": "directory","metaphor":"","role":"","notes":"","children": {}},
        "outputs": {"type": "directory","metaphor":"","role":"","notes":"","children": {}},
    })

# a) Ocean → England_1
_inject_std(structure["Leviathan"]["children"]["Ocean"]["children"]["England_1"])

# b) every leaf inside Armory/England_1
for leaf in structure["Leviathan"]["children"]["Armory"]["children"]["England_1"]["children"].values():
    _inject_std(leaf)

# c) every leaf inside Awhalin/England_1
for leaf in structure["Leviathan"]["children"]["Awhalin"]["children"]["England_1"]["children"].values():
    _inject_std(leaf)

# --- 3. create folders / placeholders ----------------------------------------
def _create(node, base):
    base.mkdir(exist_ok=True)
    for name, meta in node.get("children", {}).items():
        path = base / name
        if meta["type"] == "directory":
            path.mkdir(exist_ok=True)
            _create(meta, path)
        elif meta["type"] == "file":
            path.parent.mkdir(parents=True, exist_ok=True)
            if path.suffix == ".ipynb" and not path.exists():
                empty_nb = {"cells": [], "metadata": {}, "nbformat": 4, "nbformat_minor": 5}
                path.write_text(json.dumps(empty_nb, indent=2))
            else:
                path.touch(exist_ok=True)

# write YAML + scaffold
MANIFEST.write_text(yaml.safe_dump(structure, sort_keys=False))
_create(structure["Leviathan"], ROOT)
print("✅ YAML manifest updated & folders scaffolded.")


✅ YAML manifest updated & folders scaffolded.


In [55]:
def _validate(node, base):
    """Check for missing items *and* flag extraneous ones."""
    ok = True
    expected = set(node.get("children", {}).keys())

    # 1 missing items
    for name, meta in node.get("children", {}).items():
        path = base / name
        if meta["type"] == "directory":
            if not path.is_dir():
                print(f"❌ missing folder {path.relative_to(ROOT)}"); ok = False
            ok &= _validate(meta, path)
        elif meta["type"] == "file":
            if not path.is_file():
                print(f"❌ missing file   {path.relative_to(ROOT)}"); ok = False

    SKIP = {".git", ".github", ".ipynb_checkpoints"}
    
    for child in base.iterdir():
        if child.name not in expected and child.name not in SKIP:
            print(f"⚠️ extraneous item {child.relative_to(ROOT)}")

    return ok

# 👉 run the check, starting at the top of the tree
passed = _validate(structure["Leviathan"], ROOT)
print("\n✅ structure matches manifest." if passed else "\n⚠️  mismatch detected.")

⚠️ extraneous item Pequod\Chartroom\Logbook\Log.url
⚠️ extraneous item Nantucket\directory_glossary.md

✅ structure matches manifest.


In [58]:
import shutil, pathlib, os, stat

ROOT = NOTEBOOK_DIR.parent

def _force_remove(func, path, exc):
    """Callback for rmtree: clear read-only and retry."""
    os.chmod(path, stat.S_IWRITE)
    func(path)

def purge(items):
    for rel in items:
        target = ROOT / rel
        if not target.exists():
            print(f"⚠ {rel} not found"); continue
        try:
            if target.is_dir():
                shutil.rmtree(target, onerror=_force_remove)
            else:
                os.chmod(target, stat.S_IWRITE)
                target.unlink()
            print(f"🗑️ removed {rel}")
        except PermissionError as e:
            print(f"❌ couldn't delete {rel}: {e}")

# Example call
purge([
    # obsolete renders & temp files in Nantucket
    "Nantucket/directory_glossary.md",


])

⚠ Nantucket/directory_glossary.md not found
